<a href="https://colab.research.google.com/github/rssubramaniyan1/EVA8/blob/main/EVA8_Assignment4_Attempt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Attempt 3**

**Target:**

> Include GAP and FC layer to reduce the no of params compared to Ateempt 2

> Tweak the architechture from attempt 2 to try and achieve higher accuracy despite lower no of params

> Include the scheduler for changing the LR rate

________________________________________________________________________________
**Results:**

> Parameters: **6134**

> Best Training Accuracy: **98.6717 (EPOCH 14)**

> Best Test Accuracy: **99.27 (EPOCH 14)**

________________________________________________________________________________

**Analysis:**

>* Increase a layer in convolution block-2 for a higher receptive field before the second max pool layer 

>* Addition of two fc layers post the gap layer

>* Despite a **decrease of 33% in the parameters**  from attemp 2achieved accuracy of 99.27%

>* **99.27 accuracy in epoch 14** with epochs 10,11,12,13 having accuracy of 99.06,  99.22,99.13 and 99.09 respectively





In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from torch.optim import lr_scheduler
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

In [2]:

torch.manual_seed(1)
batch_size = 128
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                     transform=transforms.Compose([
                        transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=batch_size, shuffle=True, **kwargs)


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)

        # Init
        optimizer.zero_grad()
        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.nll_loss(y_pred, target)
        #train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.4f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [3]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # CONV BLOCK 1

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),  # input size 28x28x1, output size 28x28x8, RF 3x3
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),

            nn.Conv2d(8, 10, 3, padding=1),  # input size 28x28x8, output size 28x28x10, RF 5x5
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.1),

            nn.Conv2d(10, 12, 3, padding=1),  # input size 28x28x10, output size 28x28x12, RF 7x7
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1)

        )

        # MAX POOL - 1

        self.pool1 = nn.MaxPool2d(2, 2)  # input size 28x28x12, output size 14x14x12, RF 7x7
        
        # CONV BLOCK 2

        self.conv2 = nn.Sequential(
            nn.Conv2d(12, 8, 1, padding=0),  # input size 14x14x12, output size 14x14x8, RF 14x14
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),

            nn.Conv2d(8, 10, 3, padding=0),   # input size 14x14x8, output size 12x12x10, RF 16x16
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.1),

            nn.Conv2d(10, 12, 3, padding=1), # input size 12x12x10, output size 12x12x12, RF 18x18
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1)

        )
        self.pool2 = nn.MaxPool2d(2, 2)   # input size 12x12x12, output size 6x6x12, RF 18x18

        self.conv3 = nn.Sequential(
            nn.Conv2d(12, 16, 3, padding=0),    # input size 6x6x12, output size 4x4x16, RF 36x36
            nn.ReLU(),
            # nn.BatchNorm2d(16),
            # nn.Dropout(0.1),
        )

        self.gap = nn.AvgPool2d(kernel_size=3) # input size 4x4x16, output size 1x1x16, RF 36x36

        self.fc1 = nn.Linear(16, 16)  # input size 16, output size 16
        self.fc2 = nn.Linear(16, 10)  # input size 16, output size 10

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.gap(x)
        x = x.view(-1, 16)
        x = self.fc1(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 10, 28, 28]             730
              ReLU-6           [-1, 10, 28, 28]               0
       BatchNorm2d-7           [-1, 10, 28, 28]              20
           Dropout-8           [-1, 10, 28, 28]               0
            Conv2d-9           [-1, 12, 28, 28]           1,092
             ReLU-10           [-1, 12, 28, 28]               0
      BatchNorm2d-11           [-1, 12, 28, 28]              24
          Dropout-12           [-1, 12, 28, 28]               0
        MaxPool2d-13           [-1, 12, 14, 14]               0
           Conv2d-14            [-1, 8,

In [4]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

for epoch in range(1, 15):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

EPOCH: 1


loss=0.21720880270004272 batch_id=468 Accuracy=80.1200: 100%|██████████| 469/469 [00:36<00:00, 12.75it/s]



Test set: Average loss: 0.0965, Accuracy: 9699/10000 (96.9900%)

EPOCH: 2


loss=0.07858528941869736 batch_id=468 Accuracy=96.4367: 100%|██████████| 469/469 [00:37<00:00, 12.51it/s]



Test set: Average loss: 0.0511, Accuracy: 9831/10000 (98.3100%)

EPOCH: 3


...


Test set: Average loss: 0.0291, Accuracy: 9913/10000 (99.1300%)

EPOCH: 13


loss=0.06926364451646805 batch_id=468 Accuracy=98.6017: 100%|██████████| 469/469 [00:37<00:00, 12.48it/s]



Test set: Average loss: 0.0256, Accuracy: 9909/10000 (99.0900%)

EPOCH: 14


loss=0.07597482949495316 batch_id=468 Accuracy=98.6717: 100%|██████████| 469/469 [00:37<00:00, 12.50it/s]



Test set: Average loss: 0.0235, Accuracy: 9927/10000 (99.2700%)

